# Remove certain node types from problem graphs

In [6]:
%env CUDA_VISIBLE_DEVICES=""

env: CUDA_VISIBLE_DEVICES=""


TODO:
    
    My goal is to create code which removes the nodes of a given type

TODO - what about premise labels??

In [7]:
def construct_graph(nodes, sources, targets):
    net = Network(notebook=True, directed=True)

    for i, n in enumerate(nodes):
        net.add_node(i, label=NODE_TYPE[n], group=int(n))

    for s, t in zip(sources, targets):
        net.add_edge(int(s), int(t))
    return net

In [8]:
import sys
import os
from pathlib import Path
import networkx as nx
import numpy as np
import seaborn as sns
from pyvis.network import Network

sys.path.append(str(Path(os.path.abspath("")).parent))
from dataset import load_ids
from graph_parser import graph
from itertools import product


from config import NODE_TYPE
import config
from read_problem import read_problem_deepmath
from collections import Counter

In [9]:
# ID_FILE = '../id_files/deepmath.txt'
ID_FILE = "../id_files/dev_100.txt"
# ID_FILE = '../id_files/validation.txt'

In [10]:
ids = load_ids(ID_FILE)

In [11]:
axiom = [b"fof(parent,axiom, ( child(X, Y)))."]
conjecture = []

nodes, sources, targets, premise_indices, conjecture_indices = graph(conjecture, axiom)

In [12]:
net_conj = construct_graph(nodes, sources, targets)
net_conj.show("graph_conjecture.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


# Function from here!

In [13]:
def remove_node_type(nodes, sources, targets, premise_indices, conjecture_indices, node_type=4):

    # Check if node type exists in the graph
    if node_type not in nodes:
        return new_nodes, new_sources, new_targets, new_premise_indices, new_conjecture_indices

    # Convert into a networkx Directional graph
    G = nx.DiGraph()
    G.add_edges_from(list(zip(sources, targets)))

    # Transfer the node types
    attr = {i: t for i, t in enumerate(nodes)}
    nx.set_node_attributes(G, attr, name="type")

    # Remove the given node type
    for node_id in list(G.nodes):

        # Check if node is of the correct type
        if G.nodes[node_id]["type"] != node_type:
            continue

        # Get all in/out edges and remap current node
        in_nodes = [a for a, b in G.in_edges(node_id)]
        out_nodes = [b for a, b in G.out_edges(node_id)]

        new_edges = list(product(*[in_nodes, out_nodes]))
        G.add_edges_from(new_edges)

        # Finally remove the node
        G.remove_node(node_id)

    # Remap and restructure indices to get it back to the original format
    node_map = {i: n for n, i in enumerate(list(G.nodes))}
    map_node = {n: i for n, i in enumerate(list(G.nodes))}

    new_targets = [t for s, t in G.edges]
    new_targets = list(map(node_map.get, new_targets))

    new_sources = [s for s, t in G.edges]
    new_sources = list(map(node_map.get, new_sources))

    new_nodes = [G.nodes[map_node[n]]["type"] for n in range(len(map_node))]

    new_premise_indices = list(map(node_map.get, premise_indices))
    new_conjecture_indices = list(map(node_map.get, conjecture_indices))

    return new_nodes, new_sources, new_targets, new_premise_indices, new_conjecture_indices

In [14]:
new_nodes, new_sources, new_targets, new_premise_indices, new_conjecture_indices = remove_node_type(
    nodes, sources, targets, premise_indices, conjecture_indices
)

In [15]:
net_conj = construct_graph(new_nodes, new_sources, new_targets)
net_conj.show("graph_conjecture_reduced.html")

Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 
